# Commerce Challenge 3 - Baseline Submission

This notebook provides a simple baseline for **Commerce Challenge 3: Coupon Redemption Prediction**.

**Goal**: Predict `target_redeem` (0/1) for each coupon offer
**Metric**: Macro-F1 Score - Higher is better

## Instructions:
1. **Replace API credentials** in the first cell with your team's API key and name
2. **Run all cells** to generate and submit baseline predictions
3. **Check the output** for your submission score

This baseline uses only tabular coupon data with a simple Random Forest classifier.


In [ ]:
# 1. Initialize Client and Load Data

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from agentds import BenchmarkClient

# 🔑 REPLACE WITH YOUR CREDENTIALS
client = BenchmarkClient(
    api_key="your-api-key-here",        # Get from your team dashboard
    team_name="your-team-name-here"     # Your exact team name
)

# Load data from PVC paths
print("📂 Loading Commerce Challenge 3 data...")

# Load coupon offers data
train_coupons = pd.read_csv("/home/jovyan/shared/datasets/Commerce/coupon_offers_train.csv")
test_coupons = pd.read_csv("/home/jovyan/shared/datasets/Commerce/coupon_offers_test.csv")

print(f"✅ Data loaded:")
print(f"   Train coupons: {train_coupons.shape}")
print(f"   Test coupons: {test_coupons.shape}")
print(f"   Train columns: {list(train_coupons.columns)}")
print(f"   Test columns: {list(test_coupons.columns)}")


In [ ]:
# 2. Tabular-Only Baseline Model and Predictions

# From data inspection - coupon_offers columns:
# offer_id, customer_id, sku_id, category, discount_pct, price_tier, hist_spend, email_open_rate, avg_basket_value, target_redeem

# Select numeric features for baseline
numeric_features = ['customer_id', 'sku_id', 'discount_pct', 'hist_spend', 'email_open_rate', 'avg_basket_value']
print(f"📊 Using coupon features: {numeric_features}")

# Prepare training data
X_train = train_coupons[numeric_features].fillna(0)
y_train = train_coupons['target_redeem']  # Binary target (0/1)

# Prepare test data
X_test = test_coupons[numeric_features].fillna(0)

# Train simple Random Forest baseline
print("🤖 Training Random Forest classifier...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Create submission file (format: offer_id,target_redeem)
submission_df = pd.DataFrame({
    'offer_id': test_coupons['offer_id'],
    'target_redeem': predictions
})

# Save predictions
submission_df.to_csv("commerce_challenge3_predictions.csv", index=False)
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Redemption rate: {predictions.mean():.3f} ({predictions.sum()} redeemed out of {len(predictions)})")


In [ ]:
# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Commerce", 3, "commerce_challenge3_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. You've completed all Commerce challenges!")
